In [1]:
### Template for Gaussian process using the binary data sets of Co, Cu, Fe @ Zr and reaction conditions ###

In [14]:
# Import Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
tf.__version__

'2.12.1'

In [15]:
#np.random.seed(404)
np.set_printoptions(precision=2, suppress=True)

In [16]:
# Read data
data = pd.read_excel('Modelling_Data_Phase_1.xlsx', sheet_name ='Seed', index_col=False)
data.head()

,Zr,Cu,Co,Fe,STY
0,0.129,0.0,0.000,0.871,236.682003
1,0.135,0.0,0.185,0.680,294.040805
2,0.115,0.0,0.315,0.570,281.273559
3,0.117,0.0,0.460,0.423,243.321402
4,0.125,0.0,0.607,0.268,161.815333


In [100]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31 entries, 0 to 30
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Zr      31 non-null     float64
 1   Cu      31 non-null     float64
 2   Co      31 non-null     float64
 3   Fe      31 non-null     float64
 4   STY     31 non-null     float64
dtypes: float64(5)
memory usage: 1.3 KB


In [101]:
# General stastistical data
data.describe()

,Zr,Cu,Co,Fe,STY
count,31.000000,31.00000,31.000000,31.000000,31.000000
mean,0.143742,0.24529,0.236065,0.374903,171.513337
std,0.097154,0.26235,0.257175,0.314445,100.103920
min,0.000000,0.00000,0.000000,0.000000,0.000000
25%,0.113500,0.00000,0.000000,0.000000,86.619788
50%,0.122000,0.20200,0.177000,0.423000,161.547789
75%,0.142000,0.41300,0.432000,0.658000,249.106089
max,0.438000,0.89100,0.851000,0.871000,324.248728


In [102]:
# Define X and y from the data

X = data[['Zr ', 'Cu ', 'Co ', 'Fe']].values
Y = -1*data['STY'].values.reshape(-1, 1) 

In [103]:
Y

array([[-236.682],
       [-294.041],
       [-281.274],
       [-243.321],
       [-161.815],
       [ -99.863],
       [  -0.   ],
       [ -86.442],
       [-134.092],
       [-210.049],
       [-299.991],
       [-309.924],
       [  -0.   ],
       [ -90.257],
       [ -86.798],
       [ -91.53 ],
       [ -82.638],
       [ -83.368],
       [-324.249],
       [-218.904],
       [ -63.576],
       [-140.567],
       [ -71.581],
       [-161.548],
       [-239.702],
       [-238.423],
       [-254.891],
       [-304.157],
       [-321.494],
       [-121.7  ],
       [ -64.038]])

In [104]:
## Gaussian process regression 

from gpflow.models import GPR
from gpflow.models import SVGP
from gpflow.likelihoods import Gaussian
from gpflow.optimizers import Scipy
from gpflow.kernels import SquaredExponential as SE, Constant as C, White as W, SharedIndependent as SI
from gpflow.inducing_variables import SharedIndependentInducingVariables as SIIV, InducingPoints as IP
from sklearn.metrics import r2_score, mean_squared_error

In [105]:
# Kernel (signal noise * SE + data noise)

#single objective with 4 metals
kernel = SE(lengthscales=[0.1,1.3,0.1,0.3])

# Gaussian process regression
gp_model = GPR((X, Y), kernel=kernel)

# Optimization
opt = Scipy()
opt.minimize(gp_model.training_loss, gp_model.trainable_variables)


  message: CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
  success: True
   status: 0
      fun: 164.3267665107328
        x: [-1.771e+00 -2.899e-01  7.751e+00 -5.158e-01  1.184e+04
             5.951e+02]
      nit: 68
      jac: [-1.765e+00  2.724e+00  9.629e-03  2.894e+00 -1.889e-04
            -5.023e-04]
     nfev: 83
     njev: 83
 hess_inv: <6x6 LbfgsInvHessProduct with dtype=float64>

In [106]:
from sklearn.metrics import r2_score, mean_squared_error

# After optimization, you can make predictions with the trained model
Y_pred_mean, _ = gp_model.predict_y(X)  # Predicted mean values

# Evaluate R^2 score
r2 = r2_score(Y, Y_pred_mean)

# Evaluate mean squared error (RMSE)
mse = mean_squared_error(Y, Y_pred_mean)
rmse = np.sqrt(mse)

# Print the evaluation metrics with limited decimal places
print("R^2 score: {:.2f}".format(r2))
print("Root Mean Squared Error: {:.2f}".format(rmse))

R^2 score: 0.96
Root Mean Squared Error: 18.96


In [107]:
# Optimized kernel parameters

from gpflow.utilities import print_summary
gp_model.kernel.parameters


(<Parameter: name=softplus, dtype=float64, shape=[4], fn="softplus", numpy=array([0.157, 0.559, 7.752, 0.468])>,
 <Parameter: name=softplus, dtype=float64, shape=[], fn="softplus", numpy=11836.348723965033>)

In [108]:
# You can also access the optimized hyperparameters
optimized_lengthscales = gp_model.kernel.lengthscales.numpy()
print("Optimized Lengthscales:", optimized_lengthscales)

Optimized Lengthscales: [0.157 0.559 7.752 0.468]


# Bayesian optimization

In [93]:
# Defining the Constraints

from trieste.space import LinearConstraint
from trieste.space import Box

# Define lower and upper bounds for metal fractions
Zr_lb = 0.1
Zr_ub = 0.1
Cu_lb = 0.05
Cu_ub = 0.2
Co_lb = 0.05
Co_ub = 0.5
Fe_lb = 0.4
Fe_ub = 0.8

const_lb = -10
const_ub = 10


# Define linear constraints. Apply lb and ub to the scalar product of the number vector and the feature vector
# Metal compositions 

constraints = [LinearConstraint(A=tf.constant
        ([[1, 1, 1, 1], [1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]]), 
        lb=tf.constant([1, Zr_lb, Cu_lb, Co_lb, Fe_lb]), 
        ub=tf.constant([1, Zr_ub, Cu_ub, Co_ub, Fe_ub]))]
constrained_search_space = Box([0, 0, 0, 0], [1, 1, 1, 1], constraints=constraints)


In [94]:
# Functions for formatting data

from trieste.data import Dataset

def observer(in_):
    in_ = tf.convert_to_tensor(in_)
    out_, _ = gp_model.predict_y(in_)
    out_ = tf.convert_to_tensor(out_)
    return Dataset(in_, out_)

def initial_data(in_, out_):
    in_ = tf.convert_to_tensor(in_)
    out_ = tf.convert_to_tensor(out_)
    return Dataset(in_, out_)

In [95]:
# Build model

from trieste.models.gpflow import GaussianProcessRegression
from trieste.bayesian_optimizer import BayesianOptimizer
from trieste.acquisition.rule import EfficientGlobalOptimization
from trieste.acquisition.function import Fantasizer
from trieste.acquisition import LocalPenalization
from trieste.acquisition.function import ExpectedHypervolumeImprovement
from trieste.acquisition.function import ExpectedImprovement
from trieste.acquisition.function import PredictiveVariance

model = GaussianProcessRegression(gp_model, num_kernel_samples=10)

# Acquisition functions and rule

ei = ExpectedImprovement(constrained_search_space)
rule_ei = EfficientGlobalOptimization(builder=ei)

pv = PredictiveVariance()
rule_pv = EfficientGlobalOptimization(builder=pv)


# Bayesian optimizer
bo = BayesianOptimizer(observer, constrained_search_space)


In [96]:
# Run the Bayesian optimizer for single objective

batch_size = 6
bo_result = bo.optimize(batch_size, initial_data(X, Y), model, rule_ei, track_state = False, fit_initial_model=False)

Optimization completed without errors


In [97]:
# Get results from the Bayesian optimizer
bo_initial_data = bo_result.try_get_final_dataset()
bo_X = bo_result.try_get_final_dataset().query_points.numpy()[-batch_size:,:]

bo_Y = -1*bo_result.try_get_final_dataset().observations.numpy()[-batch_size:,:]
np.set_printoptions(precision=3, suppress=True)

result=(np.concatenate((bo_X, bo_Y), axis=1))


In [98]:
# Create dataframe with results 

dfresult = pd.DataFrame(result, columns = ['Zr','Cu','Co','Fe','STY'])
dfresult = dfresult.round(2)
dfresult

,Zr,Cu,Co,Fe,STY
0,0.1,0.06,0.12,0.72,329.34
1,0.1,0.05,0.11,0.73,329.25
2,0.1,0.05,0.11,0.73,329.25
3,0.1,0.05,0.11,0.73,329.26
4,0.1,0.05,0.13,0.72,329.19
5,0.1,0.05,0.13,0.72,329.19
